In [27]:
%pwd

'c:\\Users\\sam\\End-to-End-Fashion-Recommendation-System-with-MLOps'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\sam\\End-to-End-Fashion-Recommendation-System-with-MLOps'

In [28]:
from dataclasses import dataclass
import os
from pathlib import Path


In [29]:
@dataclass(frozen=True)
class Data_validation_config:
    validated_data:     Path
    fallback_data:      Path
    log_path:           Path
    expected_columns:   list[str]
    missing_thresholds: dict[str, float]
    volume:             dict[str, int]
    valid_aesthetics:   list[str]
    range_checks:       dict[str, dict]

In [30]:
from src.recommendation_system.utils.common import read_yaml , create_dir
from src.recommendation_system.logging import logger
from src.recommendation_system.constants import CONFIG_PATH

In [39]:
class Config_manager:

    def __init__(self, config_path = CONFIG_PATH):
        self.config_path = read_yaml(config_path)

        create_dir([self.config_path.artifacts_root])

    
    def get_data_validation_config(self) -> Data_validation_config:

        config = self.config_path.data_validation

        create_dir([config.validated_data , 
                    config.fallback_data,
                    os.path.dirname(self.config.log_path)])
        
        data_validation_config = Data_validation_config(
            validated_data     = config.validated_data,
        fallback_data      = config.fallback_data,
        log_path           = config.log_path,
        expected_columns   = config.expected_columns,
        missing_thresholds = dict(config.missing_thresholds),
        volume             = dict(config.volume),
        valid_aesthetics   = config.valid_aesthetics,
        range_checks       = dict(config.range_checks),
)
        return data_validation_config



        

In [ ]:
class Data_validation_check:

    def __init__(self , config : Data_validation_config):
        
        self.config = config

        #####n step 1 check schema 

    def check_schema(self , df):

        expected_colums = self.config.expected_columns
        recieved_columns = list(df.columns)

        missing_columns = [col for col in expected_colums if col not in recieved_columns ]
        extra_columns = [col for col in recieved_columns if col not in expected_colums ]  

        return missing_columns ,extra_columns 

In [ ]:
con = Config_manager()
data_ingestion = con.get_data_validation_config()
data_ingestion = Data_validation_check(data_ingestion)
data_ingestion.che()